In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

# Access Data via API

In [2]:
conn=http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET", "/summary", payload, headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')

### Convert Data to JSON

In [3]:
covid1=json.loads(data)

# Normalize Data

In [4]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,0,51526,0,2191,0,41727,2021-01-02T23:29:42Z
1,Albania,AL,albania,0,58316,0,1181,0,33634,2021-01-02T23:29:42Z
2,Algeria,DZ,algeria,287,99897,6,2762,268,67395,2021-01-02T23:29:42Z
3,Andorra,AD,andorra,68,8117,0,84,31,7463,2021-01-02T23:29:42Z
4,Angola,AO,angola,15,17568,0,405,102,11146,2021-01-02T23:29:42Z
...,...,...,...,...,...,...,...,...,...,...
187,Viet Nam,VN,vietnam,9,1474,0,35,0,1325,2021-01-02T23:29:42Z
188,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2021-01-02T23:29:42Z
189,Yemen,YE,yemen,2,2101,0,610,2,1396,2021-01-02T23:29:42Z
190,Zambia,ZM,zambia,272,20997,2,390,113,18773,2021-01-02T23:29:42Z


# Convert to Pandas DataFrame

In [5]:
df=pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,0,51526,0,2191,0,41727,2021-01-02T23:29:42Z,{}
1,Albania,AL,albania,0,58316,0,1181,0,33634,2021-01-02T23:29:42Z,{}
2,Algeria,DZ,algeria,287,99897,6,2762,268,67395,2021-01-02T23:29:42Z,{}
3,Andorra,AD,andorra,68,8117,0,84,31,7463,2021-01-02T23:29:42Z,{}
4,Angola,AO,angola,15,17568,0,405,102,11146,2021-01-02T23:29:42Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
187,Viet Nam,VN,vietnam,9,1474,0,35,0,1325,2021-01-02T23:29:42Z,{}
188,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2021-01-02T23:29:42Z,{}
189,Yemen,YE,yemen,2,2101,0,610,2,1396,2021-01-02T23:29:42Z,{}
190,Zambia,ZM,zambia,272,20997,2,390,113,18773,2021-01-02T23:29:42Z,{}


# Drop Unnecessary Columns

In [6]:
covid2 = df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,0,51526,0,2191,0,41727
1,Albania,0,58316,0,1181,0,33634
2,Algeria,287,99897,6,2762,268,67395
3,Andorra,68,8117,0,84,31,7463
4,Angola,15,17568,0,405,102,11146
...,...,...,...,...,...,...,...
187,Viet Nam,9,1474,0,35,0,1325
188,Western Sahara,0,10,0,1,0,8
189,Yemen,2,2101,0,610,2,1396
190,Zambia,272,20997,2,390,113,18773


# Generate Base Map

In [7]:
m = folium.Map(titles = 'Stamen Terrain', min_zoom = 1.5)
m

# Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [8]:
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes= f'{url}/world-countries.json'

# Generate Choropleth Map Layer

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid2,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

# Generate Circular Markers

In [10]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

#### coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-cordinates-world.csv

In [11]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [12]:
covid_final=pd.merge(covid2,coordinates, on='Country')

In [13]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalConfirmed, point.TotalRecovered],
                       fill_color='#000000').add_to(m)

In [14]:
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m

# Generate Base Map

In [15]:
m1= folium.Map(tiles='StamenToner',min_zoom=2)
m1

# Generate Heat Map Layer

In [16]:
deaths=covid_final['TotalDeaths'].astype(float)

In [17]:
lat=covid_final['latitude'].astype(float)

In [18]:
lon=covid_final['longitude'].astype(float)

In [19]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

# Generate Circular Marker

In [ ]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country, point.TotalDeaths],
                       fill_color='#000000').add_to(m1)
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1